In [614]:
print("Bismillah Hirrahamaa Nirrraheem")

Bismillah Hirrahamaa Nirrraheem


In [615]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from dash import html,dcc,Input,Output,callback
# pd.set_option("display.float_format","{:.2f}".format)
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Fira Code",size=20)))
pio.templates.default = "plotly_dark+mod"
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedGroupKFold,StratifiedKFold,GridSearchCV
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,LabelEncoder,OneHotEncoder,MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,VotingClassifier
from zipfile import ZipFile
from glob import glob
import sys
import shutil
import warnings
warnings.filterwarnings(action="ignore")

In [616]:
%xmode Minimal
%matplotlib inline

Exception reporting mode: Minimal


In [617]:
if "google.colab" in sys.modules:
    path = "https://github.com/h4ck4l1/datasets/raw/main/fraud_detection/"
else:
    path = "/home/sohail/Downloads/archive (2) (1)/Data/"

In [618]:
all_file_names = [
    "account_activity",
    "amount_data",
    "anomaly_scores",
    "customer_data",
    "fraud_indicators",
    "merchant_data",
    "suspicious_activity",
    "transaction_category_labels",
    "transaction_metadata",
    "transaction_records"
    ]

In [619]:
len(all_file_names)

10

In [620]:
def preprocess():
    global account_activity,amount_data,anomaly_scores,customer_data,fraud_indicators,merchant_data,suspicious_activity,transaction_category_labels,transaction_metadata,transaction_records
    account_activity = pd.read_csv(path+all_file_names[0]+".csv")
    amount_data = pd.read_csv(path+all_file_names[1]+".csv")
    anomaly_scores = pd.read_csv(path+all_file_names[2]+".csv")
    customer_data = pd.read_csv(path+all_file_names[3]+".csv")
    fraud_indicators = pd.read_csv(path+all_file_names[4]+".csv")
    merchant_data = pd.read_csv(path+all_file_names[5]+".csv")
    suspicious_activity = pd.read_csv(path+all_file_names[6]+".csv")
    transaction_category_labels = pd.read_csv(path+all_file_names[7]+".csv")
    transaction_metadata = pd.read_csv(path+all_file_names[8]+".csv")
    transaction_records = pd.read_csv(path+all_file_names[9]+".csv")
    df = pd.merge(left=account_activity,right=customer_data,right_on="CustomerID",left_on="CustomerID")
    df = pd.merge(left=df,right=transaction_records,left_on="CustomerID",right_on="CustomerID")
    df = pd.merge(left=df,right=suspicious_activity,left_on="CustomerID",right_on="CustomerID")
    df = pd.merge(left=df,right=transaction_metadata,left_on="TransactionID",right_on="TransactionID")
    df = pd.merge(left=df,right=amount_data,left_on="TransactionID",right_on="TransactionID")
    df = pd.merge(left=df,right=fraud_indicators,left_on="TransactionID",right_on="TransactionID")
    df = pd.merge(left=df,right=anomaly_scores,left_on="TransactionID",right_on="TransactionID")
    df = pd.merge(left=df,right=transaction_category_labels,left_on="TransactionID",right_on="TransactionID")
    df = pd.merge(left=df,right=merchant_data,left_on="MerchantID",right_on="MerchantID")
    df.drop(['Name','Address','MerchantName','Location','LastLogin','TransactionID','MerchantID','CustomerID'],axis=1,inplace=True)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    return df

In [621]:
df = preprocess()

<font size=4>

|Column Name|Type of column|Column Description|
|------|-----|-----|
|Customer ID|Categorical|Value counts will tell us how many transactions they have done|
|Account Balance|Continuous|The Amount of money left in their bank account|
|Age|Continuous|Ages at which they made a transactions|
|Transaction ID|Categorical|Unique IDs given to transactions|
|Transaction Amount|Continuous|Amount of the transactions thats carried out|
|Suspicious Flag|Categorical|Suspicious flag 0 or 1, 0 for not 1 for yes|
|Timestamp|Continuous|Time at which transaction has been carried out|
|Merchant ID|Categorical|Unique ID at the the transaction has been carried out|
|Amount|Continuous|Transaction Amount of the fraudulent activity|
|Fraud Indicator|Categorical|Whether its been flagged Fraud or not|
|Anomaly Score|Continuous|The score given for its potential fraud|
|Category|Categorical|Categories at which they made the transactions|

</font>

In [622]:
time_index = pd.date_range(start=df["Timestamp"].min(),end=df["Timestamp"].max(),freq="H")
display(time_index[0])
time_index[-1]

Timestamp('2022-01-01 00:00:00')

Timestamp('2022-02-11 15:00:00')

In [623]:
df.head()

,AccountBalance,Age,Amount,SuspiciousFlag,Timestamp,TransactionAmount,FraudIndicator,AnomalyScore,Category
0,9507.272060,54,33.670626,0,2022-02-02 18:00:00,87.375444,0,0.030921,Food
1,7268.827350,37,23.740319,0,2022-01-11 16:00:00,71.337967,0,0.555628,Food
2,1715.321989,40,30.980239,0,2022-01-27 16:00:00,46.044172,0,0.961552,Travel
3,3101.509134,30,11.167088,0,2022-01-19 17:00:00,85.085061,0,0.525809,Other
4,6847.490921,36,16.182847,1,2022-01-26 12:00:00,33.420638,0,0.260799,Other


In [624]:
temp = df.copy()
temp = temp.set_index('Timestamp').sort_index()
temp["weekday"] = temp.index.day_name()
temp["Hour"] = temp.index.strftime("%H")
temp["Working"] = np.NaN
temp.loc[temp.between_time(start_time="9:00:00",end_time="17:00:00").index,"Working"] = 1
temp.fillna(0,inplace=True)
temp["Day"] = np.NaN
temp.loc[temp.between_time(start_time="6:00:00",end_time="18:00:00").index,"Day"] = 1
temp.fillna(0,inplace=True)
temp.head()

,AccountBalance,Age,Amount,SuspiciousFlag,TransactionAmount,FraudIndicator,AnomalyScore,Category,weekday,Hour,Working,Day
Timestamp,,,,,,,,,,,,
2022-01-01 00:00:00,2869.689912,50,55.530334,0,79.413607,0,0.686699,Other,Saturday,00,0.0,0.0
2022-01-01 01:00:00,9527.947107,46,12.881180,0,12.053087,0,0.081749,Online,Saturday,01,0.0,0.0
2022-01-01 02:00:00,9288.355525,34,50.176322,0,33.310357,0,0.023857,Travel,Saturday,02,0.0,0.0
2022-01-01 03:00:00,5588.049942,33,41.634001,0,46.121117,0,0.876994,Travel,Saturday,03,0.0,0.0
2022-01-01 04:00:00,7324.785332,18,78.122853,0,54.051618,0,0.034059,Other,Saturday,04,0.0,0.0


# Univariate Tests

## Continuous Variables

### Account Balance

In [625]:
px.histogram(temp,x="AccountBalance",marginal="violin",height=700).add_vline(x=temp.AccountBalance.mean(),line=dict(dash="dash",color="#202ff5"),annotation=dict(text=f"mean = {temp.AccountBalance.mean():.2f}",y=0.25,font=dict(color="#ffffff",size=20),align="center"))

In [626]:
stats.shapiro(temp.AccountBalance)

ShapiroResult(statistic=0.9544827342033386, pvalue=4.637886551043559e-17)

In [627]:
data = sm.qqplot(temp.AccountBalance,line="s").gca().lines
plt.close()
fig = go.Figure()
fig.add_trace(go.Scatter(x=data[0].get_xdata(),y=data[0].get_ydata(),mode="markers",name="Obtained<br>Quantiles"))
fig.add_trace(go.Scatter(x=data[1].get_xdata(),y=data[1].get_ydata(),mode="lines",name="Expected<br>Quantiles"))
fig.update_layout(width=700)

By both tests 
- p value ~ 0  


we can conclude that it is not a normal distribution and by the shape of q-qplot we can confirm it is a under-dispersed data and closely resembling uniform distribution data

#### Calculating goodness of fit with Different Distributions

In [628]:
for i in ["norm","lognorm","uniform","expon","weibull_min","gamma"]:
    print(i,stats.kstest(temp.AccountBalance.to_numpy(),i,args=getattr(stats,i).fit(temp.AccountBalance))[1])

norm 3.8630340192348746e-05
lognorm 3.8609287174091645e-05
uniform 0.01762834309738495
expon 2.661086995206295e-29
weibull_min 1.9702161367259607e-05
gamma 1.8114759244118527e-05


The distribution is an unknown distribution but is a under-dispersed distribution

### Age

In [629]:
px.histogram(temp,x="Age",marginal="violin",height=700).add_vline(x=temp.Age.mean(),line=dict(dash="dash",color="#202ff5"),annotation=dict(text=f"mean = {temp.Age.mean():.2f}",y=0.25,font=dict(color="#ffffff",size=20),align="center"))

In [630]:
stats.shapiro(temp.Age)

ShapiroResult(statistic=0.9595720767974854, pvalue=5.283914344749677e-16)

In [631]:
stats.normaltest(temp.Age.to_numpy())

NormaltestResult(statistic=429.2519607880808, pvalue=6.153483519531615e-94)

In [632]:
data = sm.qqplot(temp.Age,line="s").gca().lines
plt.close()
fig = go.Figure()
fig.add_trace(go.Scatter(x=data[0].get_xdata(),y=data[0].get_ydata(),mode="markers",name="Obtained<br>Quantiles"))
fig.add_trace(go.Scatter(x=data[1].get_xdata(),y=data[1].get_ydata(),mode="lines",name="Expected<br>Quantiles"))
fig.update_layout(width=700)

Age is also following a considerable deviation normal distribution 

#### Calculating goodness of fit with different Distributions

In [633]:
for i in ["norm","lognorm","uniform","expon","weibull_min","gamma"]:
    print(i,stats.kstest(temp.Age.to_numpy(),i,args=getattr(stats,i).fit(temp.Age))[1])

norm 0.00010899743682695127
lognorm 0.00015290409372539258
uniform 0.0010621802695139622
expon 2.163856901329947e-24
weibull_min 9.964005639012219e-05
gamma 0.0002958430063155115


This distribution follows uniform at low significance and also shows normal so it can be a mixture of distributions

### Amount

In [634]:
px.histogram(temp,x="Amount",marginal="violin",height=700).add_vline(x=temp.Amount.mean(),line=dict(color="#202fff",dash='dash'),annotation=dict(text=f"mean : {temp.Amount.mean():.2f}",y=0.5))

In [635]:
data = sm.qqplot(temp.Amount.to_numpy(),line="s").gca().lines
plt.close()
fig = go.Figure()
fig.add_trace(go.Scatter(x=data[0].get_xdata(),y=data[0].get_ydata(),mode="markers",name="Observred<br>Quantiles"))
fig.add_trace(go.Scatter(x=data[1].get_xdata(),y=data[1].get_ydata(),mode="lines",name="Expeceted<br>Quantiles"))
fig.update_layout(width=700)
fig.show()

#### Testing goodness of fit with different distributions

In [636]:
for i in ["norm","lognorm","uniform","expon","weibull_min","gamma"]:
    print(i,stats.kstest(temp.Amount.to_numpy(),i,args=getattr(stats,i).fit(temp.Amount))[1])

norm 0.0008993756323730259
lognorm 0.0008993936629115325
uniform 0.16064836820301653
expon 7.513980205061292e-29
weibull_min 0.001105636328350367
gamma 0.001010926958604522


This is definitely a uniform distribution 

### Transaction Amount

In [637]:
px.histogram(temp,x="TransactionAmount",marginal="violin",height=600).add_vline(x=temp.TransactionAmount.mean(),line=dict(color="#202fff",dash='dash'),annotation=dict(text=f"mean : {temp.TransactionAmount.mean():.2f}",y=0.5))

In [638]:
stats.shapiro(temp.TransactionAmount)

ShapiroResult(statistic=0.9560242891311646, pvalue=9.487719936080355e-17)

In [639]:
stats.normaltest(temp.TransactionAmount)

NormaltestResult(statistic=563.7952795850348, pvalue=3.7446443496735614e-123)

In [640]:
for i in ["norm","expon","weibull_min","uniform","lognorm","gamma"]:
    print(i,stats.kstest(temp.TransactionAmount,i,args=getattr(stats,i).fit(temp.TransactionAmount))[1])

norm 0.0009721041164773224
expon 7.535529242910072e-26
weibull_min 0.00013305783986073338
uniform 0.28202051589421273
lognorm 0.0009725435118838702
gamma 0.0006657635113463688


This distribution closely follows a uniform distribution

## Categorical Variables

In [641]:
temp.head(2)

,AccountBalance,Age,Amount,SuspiciousFlag,TransactionAmount,FraudIndicator,AnomalyScore,Category,weekday,Hour,Working,Day
Timestamp,,,,,,,,,,,,
2022-01-01 00:00:00,2869.689912,50,55.530334,0,79.413607,0,0.686699,Other,Saturday,00,0.0,0.0
2022-01-01 01:00:00,9527.947107,46,12.881180,0,12.053087,0,0.081749,Online,Saturday,01,0.0,0.0


In [642]:
px.histogram(temp,x="SuspiciousFlag",histfunc="count",title="Suspicious Flag",height=600,color="SuspiciousFlag",color_discrete_map={0:"#636efa",1:"red"})

In [643]:
sm.stats.proportions_ztest(count=temp.SuspiciousFlag.value_counts().to_numpy(),nobs=temp.shape[0])

(42.485291572496, 0.0)

Suspicous Flag proportions are statsitically significant as p-value is 0 from one-proportions z-test

In [644]:
px.histogram(temp,x="FraudIndicator",histfunc="count",title="Fraud Indicator",height=600,color="FraudIndicator",color_discrete_map={0:"#636efa",1:"red"})

In [645]:
sm.stats.proportions_ztest(temp.FraudIndicator.value_counts(),nobs=temp.shape[0])

(40.69643719049617, 0.0)

In [646]:
temp.Category.value_counts()

Category
Other     210
Food      204
Travel    198
Online    196
Retail    192
Name: count, dtype: int64

In [647]:
fig = make_subplots(cols=2,specs=[[{"type":"xy"},{"type":"domain"}]])
fig.add_trace(go.Bar(x=temp.Category.value_counts().index,y=temp.Category.value_counts(),marker=dict(color=px.colors.qualitative.Plotly)),row=1,col=1)
fig.add_trace(go.Pie(values=temp.Category.value_counts().to_numpy(),labels=temp.Category.value_counts().index,textinfo="label+percent+value",marker=dict(colors=px.colors.qualitative.Plotly),sort=False),row=1,col=2)
fig.update_layout(height=700,showlegend=False,title="Category")
fig.update_yaxes(title="count")
fig.update_xaxes(title="category")
fig.show()

In [648]:
print("p-value  :",stats.chi2_contingency([temp.Category.value_counts(),np.full_like(temp.Category.value_counts(),1000/temp.Category.nunique())])[1])

p-value  : 0.9737704658381907


They are equal in proportion

In [649]:
px.histogram(temp,x="weekday",color="weekday",width=900,height=700,title="Weekday").update_layout(showlegend=False)

In [650]:
print("p-value  :",stats.chi2_contingency([temp.weekday.value_counts(),np.full_like(temp.weekday.value_counts(),1000/temp.weekday.nunique())])[1])

p-value  : 0.9998556130268133


In [651]:
px.histogram(temp,x="Hour",color="Hour").update_layout(showlegend=False)

In [652]:
print("p-value  :",stats.chi2_contingency([temp.Hour.value_counts(),np.full_like(temp.Hour.value_counts(),1000/temp.Hour.nunique())])[1])

p-value  : 1.0


Fraud Indicator proportions are statsitically significant as p-value is 0 from one-proportions z-test

# Double Variable Tests

## Countinous

### Between AccountBalance and TransactionAmount

In [653]:
cont_cols = ["TransactionAmount","Amount","AccountBalance","AnomalyScore","Age"]

In [654]:
ksamp_df = pd.DataFrame(columns=cont_cols,index=cont_cols)

In [655]:
for col in cont_cols:
    for ind in cont_cols:
        sample_1 = ((temp[ind]-temp[ind].min())/(temp[ind].max()-temp[ind].min())).to_numpy()
        sample_2 = ((temp[col]-temp[col].min())/(temp[col].max()-temp[col].min())).to_numpy()
        ksamp_df.loc[ind,col] = stats.ks_2samp(sample_1,sample_2).pvalue

In [656]:
np.fill_diagonal(ksamp_df.to_numpy(),0)

In [657]:
fig = px.imshow(img=ksamp_df,height=600,text_auto=".2f",color_continuous_scale="GnBu_r",title="Kolmogorov Smirnov Test")
fig.add_annotation(text="same columns values are made zero<br>to understand <br>the relative dependence",x=1.05,y=0,xanchor="left",yanchor="bottom",font=dict(size=15),xref="x domain",yref="y domain",showarrow=False)
fig.add_annotation(text="The Values are P-Values where<br>higher the value more the probability that<br>they came from same distribution<br>values greater than 0.05(significance) suggest<br>that they are indeed from same distribution",x=1.05,y=0.5,xanchor="left",yanchor="bottom",xref="x domain",yref="y domain",showarrow=False)
fig.update_yaxes(showgrid=False)
fig.update_xaxes(showgrid=False)
fig.update_layout(margin=dict(l=0,r=0))

Not a lot of dependence we can still keep the columns as the algorithms can scale better with more features without any data leakage

## Categorical Variables

### Contingency test between FraudIndicator and the rest of the categorical columns

In [658]:
temp.columns

Index(['AccountBalance', 'Age', 'Amount', 'SuspiciousFlag',
       'TransactionAmount', 'FraudIndicator', 'AnomalyScore', 'Category',
       'weekday', 'Hour', 'Working', 'Day'],
      dtype='object')

In [659]:
def givemat(col):
    return pd.crosstab(index=temp.FraudIndicator,columns=temp[col])

In [660]:
givemat("Category")

Category,Food,Online,Other,Retail,Travel
FraudIndicator,,,,,
0,195,186,200,183,191
1,9,10,10,9,7


In [661]:
print(f'''p-value for Working :{sm.stats.mcnemar(givemat("Working").to_numpy()).pvalue}''')
print(f'''p-value for Hour :{stats.chi2_contingency(givemat("Hour").to_numpy()).pvalue}''')
print(f'''p-value for Day :{sm.stats.mcnemar(givemat("Day").to_numpy()).pvalue}''')
print(f'''p-value for Category :{stats.chi2_contingency(givemat("Category").to_numpy()).pvalue}''')

p-value for Working :3.687264509158562e-71
p-value for Hour :0.7958505114310169
p-value for Day :2.4846719053465266e-122
p-value for Category :0.9577050458132295


As we can see Working and Day are having dependency as they can be considered paired because of how the time dilation

In [662]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=temp.query("FraudIndicator == 1 and Working == 0").index,
    y=temp.query("FraudIndicator == 1 and Working == 0").Amount,
    marker=dict(size=temp.query("FraudIndicator == 1 and Working == 0").AnomalyScore,sizeref=0.03,sizemin=4,color="blue",line_color="white"),
    mode="markers",
    name="Fraudulent at \n<br>Non-Working Hours\n<br>5 PM to 8 AM"
    ))
fig.add_trace(go.Scatter(
    x=temp.query("FraudIndicator == 1 and Working == 1").index,
    y=temp.query("FraudIndicator == 1 and Working == 1").Amount,
    marker=dict(size=temp.query("FraudIndicator == 1 and Working == 1").AnomalyScore,sizeref=0.03,sizemin=4,color="red",line_color="white"),
    mode="markers",
    name="Fraudulent at \n<br>Working Hours\n<br>8 AM to 5 PM"
    ))
fig.add_trace(go.Scatter(
    x=temp.query("FraudIndicator == 0 and Working == 0").index,
    y=temp.query("FraudIndicator == 0 and Working == 0").Amount,
    marker=dict(size=temp.query("FraudIndicator == 0 and Working == 0").AnomalyScore,sizeref=0.03,sizemin=4,color="blue",opacity=0.3),
    mode="markers",
    name="Non-Fraudulent at \n<br>Non-Working Hours\n<br>5 PM to 8 AM",
    visible="legendonly"
))
fig.add_trace(go.Scatter(
    x=temp.query("FraudIndicator == 0 and Working == 1").index,
    y=temp.query("FraudIndicator == 0 and Working == 1").Amount,
    marker=dict(size=temp.query("FraudIndicator == 0 and Working == 1").AnomalyScore,sizeref=0.03,sizemin=4,color="red",opacity=0.3),
    mode="markers",
    name="Non-Fraudulent at \n<br>Working Hours\n<br>8 AM to 5 PM",
    visible="legendonly"
))
fig.add_annotation(text="*Toggle normal to see <br>the non-fraudulent transactions<br> that took place",x=1.20,y=-0.10,xanchor="right",yanchor="bottom",showarrow=False,xref="x domain",yref="y domain")
fig.add_annotation(text="*size indicates AnomalyScore",x=1.20,y=0,xanchor="right",yanchor="bottom",showarrow=False,xref="x domain",yref="y domain")
fig.update_layout(title=dict(text="Fraudulent Activity by Time from January 1,2022 to February 11,2022",font=dict(size=30)),height=800,legend=dict(font_size=15,itemsizing="constant"))
fig.update_xaxes(title=dict(text="Time",font=dict(size=20)),tickfont=dict(size=20))
fig.update_yaxes(title=dict(text="Amount",font=dict(size=20)))
fig.show()

##### There are quite a bit of Fraudulent Transactions that took place at Non-Working hours

Fraudulent Transactions percentage at Non working Hours on total transactions :

In [663]:
temp.query("FraudIndicator == 1 and Working == 0").shape[0]/temp.query("FraudIndicator == 1").shape[0]

0.7555555555555555

Fraudulent Transactions percentage at Working Hours on total transactions :

In [664]:
temp.query("FraudIndicator == 1 and Working == 1").shape[0]/temp.query("FraudIndicator == 1").shape[0]

0.24444444444444444

# Modelling

In [665]:
temp.head()

,AccountBalance,Age,Amount,SuspiciousFlag,TransactionAmount,FraudIndicator,AnomalyScore,Category,weekday,Hour,Working,Day
Timestamp,,,,,,,,,,,,
2022-01-01 00:00:00,2869.689912,50,55.530334,0,79.413607,0,0.686699,Other,Saturday,00,0.0,0.0
2022-01-01 01:00:00,9527.947107,46,12.881180,0,12.053087,0,0.081749,Online,Saturday,01,0.0,0.0
2022-01-01 02:00:00,9288.355525,34,50.176322,0,33.310357,0,0.023857,Travel,Saturday,02,0.0,0.0
2022-01-01 03:00:00,5588.049942,33,41.634001,0,46.121117,0,0.876994,Travel,Saturday,03,0.0,0.0
2022-01-01 04:00:00,7324.785332,18,78.122853,0,54.051618,0,0.034059,Other,Saturday,04,0.0,0.0


In [666]:
def week(x):
    if x <= 7:
        return "week 1"
    elif (x > 7) & (x <= 14):
        return "week 2"
    elif (x > 14) & (x <= 21):
        return "week 3"
    elif (x > 21) & (x <= 28):
        return "week 4"
    else:
        return "week 5"

def Dayofmonth(x):
    if x <= 15:
        return "first half"
    else:
        return "second half"
    
def weekend(x):
    if x > 4:
        return "weekend"
    else:
        return "weekday"

In [667]:
class AppendTimeColumns(BaseEstimator,TransformerMixin):


    def fit(self,X:pd.DataFrame,y=None):
        X['Timestamp'] = pd.to_datetime(X['Timestamp'])
        X.set_index('Timestamp',inplace=True)
        X.sort_index(inplace=True)
        X["Working"] = np.NaN
        X["Day"] = np.NaN
        X["Hour"] = np.NaN
        return self
    
    def transform(self,X:pd.DataFrame,y=None):
        X["Hour"] = X.index.strftime("%H")
        X.loc[X.between_time(start_time="9:00:00",end_time="17:00:00").index,"Working"] = 1
        X["Working"] = X.Working.fillna(0)
        X.loc[X.between_time(start_time="6:00:00",end_time="18:00:00").index,"Day"] = 1
        X["Day"] = X.Day.fillna(0)
        X["Week"] = X.index.strftime('%d').astype(np.float32)
        X["day_of_week"] = X.index.dayofweek
        X["Weekday"] = X["Week"].apply(lambda x: week(x))
        X["weekend"] = X["day_of_week"].apply(lambda x: weekend(x))
        X["Dayofmonth"] = X["Week"].apply(lambda x: Dayofmonth(x))
        X["dayofweek"] = X.index.day_name()
        X.drop(['day_of_week'],inplace=True,axis=1)
        self.columns = X.columns
        return X

In [668]:
temp_df = df.copy()
y = temp_df['FraudIndicator']
temp_df.drop(['FraudIndicator'],axis=1,inplace=True)
tim = AppendTimeColumns()
transformed_df = tim.fit_transform(temp_df)

In [669]:
num_columns = ["AccountBalance","Age","Amount","TransactionAmount","AnomalyScore"]
cat_columns = ["Category","Week","Weekday","weekend","Dayofmonth","dayofweek"]

In [670]:
full_pipe = ColumnTransformer([
    ("num",MinMaxScaler(),num_columns),
    ("cat",OneHotEncoder(),cat_columns)
])

In [671]:
X = full_pipe.fit_transform(transformed_df)

In [672]:
strat = StratifiedShuffleSplit(n_splits=3,test_size=0.2,random_state=44)
grid_params = {
    "n_estimators":np.arange(100,500,step=50),
    "bootstrap":[True,False],
    "max_depth":np.arange(3,16),
    "criterion":["gini","entropy","log-loss"]
    }
grid = GridSearchCV(RandomForestClassifier(),grid_params,scoring="accuracy",n_jobs=-1,cv=strat)
grid.fit(X,y)
rand = grid.best_estimator_
strati = StratifiedShuffleSplit(1,test_size=0.3,random_state=44)
for train_ind,test_ind in strati.split(X,y):
    X_train = X[train_ind,:]
    X_test = X[test_ind,:]
    y_train = y[train_ind]
    y_test = y[test_ind]

rand.fit(X_train,y_train)
print("score : ",rand.score(X_test,y_test))
# for num,col in sorted(zip(np.round(grid.best_estimator_.feature_importances_,4),),reverse=True):
#     print(col,":",num)

score :  0.9566666666666667


In [673]:
# grid_params = {
#     "C":np.r_[np.logspace(-6,0,num=7),0],
#     "kernel":["rbf","sigmoid"]
# }
# grid = GridSearchCV(SVC(),grid_params,scoring="accuracy",n_jobs=-1,cv=strat)
# grid.fit(X,y)
# print(grid.best_estimator_)
# print(grid.best_score_)